In [ ]:
!pip install ete3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 4.2 MB/s  0:00:02 eta 0:00:01m
  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'ete3' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'ete3'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for ete3: filename=ete3-3.1.3-py3-none-any.whl size=2273900 sha256=ae0595b80237e85b8ae6dd0e774b8255d65cde2648ba397436dd6c1f33bb4ec1
  Stored in directory: /root/.cache/pip/wheels/4f/18/8d/3800b8b1dc7a8c1954eaa48424f639b2cfc760922cc3cee479
Successfully built ete3


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from ete3 import NCBITaxa
import os

# ================================
# CONFIG
# ================================
CONFIG = {
    "TAXON_FILE": "/root/CAFA6data/Train/train_taxonomy.tsv",
    "MIN_SAMPLES_REQUIRED": 10,
    "OUTPUT_PKL": "taxon_mapping_K_Species.pkl",
    "DB_PATH": "taxon.sqlite" # ete3 file
}

def get_species_level_mapping(unique_taxids):

    ncbi = NCBITaxa()
    if not os.path.exists(CONFIG["DB_PATH"]):
      ncbi.update_taxonomy_database()

    mapping = {}
    print(f" Đang map {len(unique_taxids)} taxon ID về cấp Species...")

    valid_ids = [int(tid) for tid in unique_taxids if str(tid).isdigit()]

    try:
        # Lấy rank của tất cả ID
        ranks = ncbi.get_rank(valid_ids)

        for tid in valid_ids:
            current_id = tid

            # Nếu bản thân nó là Species rồi thì giữ nguyên
            if ranks.get(current_id) == 'species':
                mapping[str(tid)] = str(current_id)
                continue

            # Nếu không phải Species, leo cây tìm cha
            try:
                lineage = ncbi.get_lineage(tid) # Trả về list [root, ..., genus, species, strain]
                # Lấy rank của cả dòng họ
                lineage_ranks = ncbi.get_rank(lineage)

                # Tìm ID nào trong dòng họ có rank là 'species'
                species_id = None
                for anc_id in reversed(lineage):   # gần nhất trước
                  if lineage_ranks.get(anc_id) == "species":
                      species_id = anc_id
                      break

                if species_id:
                    mapping[str(tid)] = str(species_id)
                else:
                    # Trường hợp không tìm thấy species (ví dụ cấp Genus), giữ nguyên ID gốc
                    mapping[str(tid)] = str(tid)

            except ValueError:
                # ID không có trong database NCBI
                mapping[str(tid)] = str(tid)

    except Exception as e:
        print(f"⚠️ Lỗi NCBI Taxa: {e}")
        # Fallback: giữ nguyên
        for tid in unique_taxids:
            mapping[str(tid)] = str(tid)

    return mapping

def analyze_for_report_final():
    print(" PHÂN TÍCH TAXONOMY (HIERARCHICAL ROLL-UP VERSION)\n")

    # 1. LOAD DATA
    df = pd.read_csv(CONFIG["TAXON_FILE"], sep="\t", header=None, names=["Protein", "Taxon"], dtype=str)

    # --- BƯỚC MỚI: ROLL-UP ---
    unique_taxa = df["Taxon"].unique()

    # Tạo map từ ID gốc -> ID loài
    # (Bạn cần cài ete3 và có internet lần đầu để tải DB)
    taxon_map = get_species_level_mapping(unique_taxa)

    # Áp dụng map vào dataframe
    df["Taxon_Original"] = df["Taxon"]
    df["Taxon"] = df["Taxon"].map(taxon_map).fillna(df["Taxon"]) # Fillna phòng hờ

    print(f" Đã quy hoạch từ {len(unique_taxa)} ID gốc xuống còn {df['Taxon'].nunique()} ID (chủ yếu là Species).")

    # --- TỪ ĐÂY TRỞ ĐI LÀ LOGIC CŨ NHƯNG TRÊN DỮ LIỆU ĐÃ GỘP ---

    min_freq = CONFIG["MIN_SAMPLES_REQUIRED"]

    tax_counts = df["Taxon"].value_counts()
    new_dead_zone = (tax_counts < min_freq).sum()




    # Lấy top taxon
    top_taxa_series = tax_counts[tax_counts >= min_freq]
    K_FINAL = len(top_taxa_series)
    UNK_IDX = K_FINAL

    print("\n--- TẠO MAPPING ---")
    print(f"K_FINAL (Mapped Species ≥ {min_freq}) = {K_FINAL}")

    # Map: Species ID -> index model
    # Lưu ý: Taxon ở đây là Species ID
    species_to_idx = {tid: i for i, tid in enumerate(top_taxa_series.index)}

    # Pre-calculate full chain mapping for speed
    full_chain_map = {} # Original ID -> Model Index
    for orig_id in unique_taxa:
        species_id = taxon_map.get(orig_id, orig_id)
        if species_id in species_to_idx:
            full_chain_map[orig_id] = species_to_idx[species_id]
        else:
            full_chain_map[orig_id] = UNK_IDX

    # Apply
    mapped_series = (
        df["Taxon_Original"]
        .map(full_chain_map)
        .fillna(UNK_IDX)
        .astype(int)
    )

    # Tạo Series sạch sẽ (xử lý NaN và ép kiểu Int)
    mapped_series = df["Taxon_Original"].map(full_chain_map).fillna(UNK_IDX).astype(int)

    # Zip cột Protein với Series sạch đó
    prot_to_taxon_idx = dict(zip(df["Protein"], mapped_series))

    # ===== COVERAGE STATS =====
    unk_count = sum(v == UNK_IDX for v in prot_to_taxon_idx.values())
    coverage = 100 * (1 - unk_count / len(prot_to_taxon_idx))

    print(f"Taxonomy coverage after roll-up: {coverage:.2f}%")
    print(f"UNK proteins: {unk_count:,} / {len(prot_to_taxon_idx):,}")

    # Lưu file
    data_to_save = {
        "taxon_to_idx": species_to_idx,
        "prot_to_taxon_idx": prot_to_taxon_idx,
        "original_id_to_species_id": taxon_map,
        "num_taxa_classes": K_FINAL + 1,
        "min_freq_cutoff": min_freq
    }

    with open(CONFIG["OUTPUT_PKL"], "wb") as f:
        pickle.dump(data_to_save, f)

    # In thống kê so sánh
    orig_counts = df["Taxon_Original"].value_counts()
    original_dead_zone = (orig_counts < min_freq).sum()

    print("\n--- HIỆU QUẢ CỦA ROLL-UP ---")
    print(f"Số lượng ID bị loại bỏ (Dead Zone) TRƯỚC khi gộp: {original_dead_zone}")
    print(f"Số lượng ID bị loại bỏ (Dead Zone) SAU khi gộp   : {new_dead_zone}")
    print(f"👉 Chúng ta đã cứu được dữ liệu từ các taxon hiếm bằng cách gộp về cha!")

if __name__ == "__main__":
    analyze_for_report_final()

 PHÂN TÍCH TAXONOMY (HIERARCHICAL ROLL-UP VERSION)



Local taxdump.tar.gz seems up-to-date


Loading node names...
2712337 names loaded.
425670 synonyms loaded.
Loading nodes...
2712337 nodes loaded.
Linking nodes...
Tree is loaded.
Updating database: /root/.etetoolkit/taxa.sqlite ...
 2712000 generating entries... 
Uploading to /root/.etetoolkit/taxa.sqlite


Inserting synonyms:      30000 

Inserting taxid merges:  45000  

Inserting taxids:       30000  

Inserting taxids:       2710000 


 Đang map 1381 taxon ID về cấp Species...
 Đã quy hoạch từ 1381 ID gốc xuống còn 1297 ID (chủ yếu là Species).

--- TẠO MAPPING ---
K_FINAL (Mapped Species ≥ 10) = 134
Taxonomy coverage after roll-up: 96.86%
UNK proteins: 2,587 / 82,404

--- HIỆU QUẢ CỦA ROLL-UP ---
Số lượng ID bị loại bỏ (Dead Zone) TRƯỚC khi gộp: 1241
Số lượng ID bị loại bỏ (Dead Zone) SAU khi gộp   : 1163
👉 Chúng ta đã cứu được dữ liệu từ các taxon hiếm bằng cách gộp về cha!


/venv/main/lib/python3.12/site-packages/ete3/ncbi_taxonomy/ncbiquery.py:243: UserWarning: taxid 61267 was translated into 1538424
  warnings.warn("taxid %s was translated into %s" %(taxid, merged_conversion[taxid]))
/venv/main/lib/python3.12/site-packages/ete3/ncbi_taxonomy/ncbiquery.py:243: UserWarning: taxid 34647 was translated into 1879292
  warnings.warn("taxid %s was translated into %s" %(taxid, merged_conversion[taxid]))
/venv/main/lib/python3.12/site-packages/ete3/ncbi_taxonomy/ncbiquery.py:243: UserWarning: taxid 184540 was translated into 8732
  warnings.warn("taxid %s was translated into %s" %(taxid, merged_conversion[taxid]))


In [ ]:
import pickle
import random

# Đường dẫn file bạn vừa lưu
PKL_PATH = "taxon_mapping_K_Species.pkl"

def inspect_pickle_file(path):
    print(f"📂 Đang đọc file: {path}...\n")

    with open(path, "rb") as f:
        data = pickle.load(f)

    # 1. Kiểm tra các Key có trong file
    print("--- CẤU TRÚC DỮ LIỆU ---")
    print(f"Các keys có trong dictionary: {list(data.keys())}")

    # 2. Lấy thông tin thống kê
    num_classes = data['num_taxa_classes']
    cutoff = data['min_freq_cutoff']

    # Index của lớp UNK luôn là class cuối cùng (num_classes - 1)
    unk_idx = num_classes - 1

    print(f"\n--- THỐNG KÊ CƠ BẢN ---")
    print(f"🔹 Tổng số lớp (Classes) cho Model : {num_classes}")
    print(f"🔹 Index của lớp UNK (Unknown)     : {unk_idx}")
    print(f"🔹 Ngưỡng tần suất tối thiểu       : {cutoff}")

    # 3. Soi dữ liệu mapping: Protein -> Index
    prot_map = data['prot_to_taxon_idx']
    print(f"\n--- SAMPLE: PROTEIN -> MODEL INDEX ---")
    print(f"Tổng số Protein đã map: {len(prot_map):,}")

    # Lấy ngẫu nhiên 5 protein để xem
    sample_prots = random.sample(list(prot_map.items()), 5)
    for prot, idx in sample_prots:
        status = "✅ Học được" if idx != unk_idx else "⚠️ Rơi vào UNK"
        print(f"  Protein {prot:<15} -> Index {idx:<5} ({status})")

    # Đếm nhanh số lượng UNK trong map
    unk_count = list(prot_map.values()).count(unk_idx)
    print(f"👉 Tổng số protein bị gán nhãn UNK: {unk_count:,} ({unk_count/len(prot_map)*100:.2f}%)")

    # 4. Soi dữ liệu mapping: Species ID -> Index
    tax_map = data['taxon_to_idx']
    print(f"\n--- SAMPLE: SPECIES ID -> MODEL INDEX ---")
    # Lấy 5 item đầu tiên
    first_5_taxa = list(tax_map.items())[:5]
    for tax_id, idx in first_5_taxa:
        print(f"  Taxon ID {tax_id:<10} -> Index {idx}")

    # 5. Soi dữ liệu truy vết: ID Gốc -> Species ID
    trace_map = data['original_id_to_species_id']
    print(f"\n--- SAMPLE: TRA CỨU (Original -> Species) ---")
    # Tìm thử vài trường hợp ID gốc khác ID species (đã được roll-up)
    found = 0
    for orig, species in trace_map.items():
        if orig != species:
            print(f"  🔁 Đã gộp: ID gốc {orig} -> Species cha {species}")
            found += 1
            if found >= 5: break

    if found == 0:
        print("  (Không tìm thấy sample nào khác biệt trong 5 lần thử đầu - có thể data toàn species chuẩn)")

# Chạy hàm
try:
    inspect_pickle_file(PKL_PATH)
except FileNotFoundError:
    print("❌ Lỗi: Không tìm thấy file .pkl! Hãy chắc chắn bạn đã chạy bước tạo file trước đó.")

📂 Đang đọc file: taxon_mapping_K_Species.pkl...

--- CẤU TRÚC DỮ LIỆU ---
Các keys có trong dictionary: ['taxon_to_idx', 'prot_to_taxon_idx', 'original_id_to_species_id', 'num_taxa_classes', 'min_freq_cutoff']

--- THỐNG KÊ CƠ BẢN ---
🔹 Tổng số lớp (Classes) cho Model : 135
🔹 Index của lớp UNK (Unknown)     : 134
🔹 Ngưỡng tần suất tối thiểu       : 10

--- SAMPLE: PROTEIN -> MODEL INDEX ---
Tổng số Protein đã map: 82,404
  Protein Q07523          -> Index 4     (✅ Học được)
  Protein P39084          -> Index 134   (⚠️ Rơi vào UNK)
  Protein P77245          -> Index 6     (✅ Học được)
  Protein P19073          -> Index 3     (✅ Học được)
  Protein P84831          -> Index 78    (✅ Học được)
👉 Tổng số protein bị gán nhãn UNK: 2,587 (3.14%)

--- SAMPLE: SPECIES ID -> MODEL INDEX ---
  Taxon ID 9606       -> Index 0
  Taxon ID 10090      -> Index 1
  Taxon ID 3702       -> Index 2
  Taxon ID 4932       -> Index 3
  Taxon ID 10116      -> Index 4

--- SAMPLE: TRA CỨU (Original -> Species) -

In [ ]:
import pickle
import random

# ĐƯỜNG DẪN FILE CŨ (Sửa tên file nếu cần)
OLD_PKL_PATH = "/root/CAFA6data/cafa6-embeds/taxon_mapping_K140.pkl"

def inspect_old_pickle(path):
    print(f"📂 Đang đọc file CŨ: {path}...\n")

    try:
        with open(path, "rb") as f:
            data = pickle.load(f)
    except FileNotFoundError:
        print(f"❌ Không tìm thấy file: {path}")
        return

    # 1. Kiểm tra Keys
    print("--- CẤU TRÚC DỮ LIỆU ---")
    print(f"Các keys: {list(data.keys())}")

    # 2. Thống kê cơ bản
    # Dùng .get() để tránh lỗi nếu file cũ cấu trúc khác
    num_classes = data.get('num_taxa_classes', 'N/A')
    cutoff = data.get('min_freq_cutoff', 'N/A')

    unk_idx = num_classes - 1 if isinstance(num_classes, int) else -1

    print(f"\n--- THỐNG KÊ ---")
    print(f"🔹 Số lớp (Classes) : {num_classes}")
    print(f"🔹 Index UNK        : {unk_idx}")
    print(f"🔹 Cutoff tần suất  : {cutoff}")

    # 3. Phân tích độ phủ (Coverage)
    if 'prot_to_taxon_idx' in data:
        prot_map = data['prot_to_taxon_idx']
        total_prots = len(prot_map)
        unk_count = list(prot_map.values()).count(unk_idx)
        coverage = (total_prots - unk_count) / total_prots * 100

        print(f"\n--- HIỆU SUẤT MAPPING (CŨ) ---")
        print(f"Tổng Protein: {total_prots:,}")
        print(f"⛔ Rơi vào UNK: {unk_count:,} ({unk_count/total_prots*100:.2f}%)")
        print(f"✅ Coverage   : {coverage:.2f}%")

        # So sánh nhanh (Mental check)
        print(f"   (So với bản mới: Bản mới UNK chỉ khoảng 3%)")

    # 4. SOIG GƯƠNG MẶT "THỪA THÃI" (Redundancy Check)
    # Đây là phần quan trọng để thấy tại sao cách cũ dở
    if 'taxon_to_idx' in data:
        tax_map = data['taxon_to_idx']
        print(f"\n--- SOI TAXON ID (Top 10) ---")
        # In ra 10 cái đầu tiên để xem có bị trùng lặp strain không
        for i, (tid, idx) in enumerate(list(tax_map.items())[:15]):
            print(f"  ID {tid:<10} -> Index {idx}")

    # 5. Kiểm tra xem có mapping Roll-up không (Chắc chắn là không)
    if 'original_id_to_species_id' not in data:
        print(f"\n⚠️ File này KHÔNG có mapping 'original_id_to_species_id'.")
        print("👉 Nghĩa là: Model đang học ID gốc thô (Raw ID). E.coli K12 và E.coli gốc bị coi là 2 loài khác nhau.")

if __name__ == "__main__":
    inspect_old_pickle(OLD_PKL_PATH)

📂 Đang đọc file CŨ: /root/CAFA6data/cafa6-embeds/taxon_mapping_K140.pkl...

--- CẤU TRÚC DỮ LIỆU ---
Các keys: ['taxon_to_idx', 'prot_to_taxon_idx', 'num_taxa_classes', 'min_freq_cutoff']

--- THỐNG KÊ ---
🔹 Số lớp (Classes) : 141
🔹 Index UNK        : 140
🔹 Cutoff tần suất  : 10

--- HIỆU SUẤT MAPPING (CŨ) ---
Tổng Protein: 82,404
⛔ Rơi vào UNK: 2,719 (3.30%)
✅ Coverage   : 96.70%
   (So với bản mới: Bản mới UNK chỉ khoảng 3%)

--- SOI TAXON ID (Top 10) ---
  ID 9606       -> Index 0
  ID 10090      -> Index 1
  ID 3702       -> Index 2
  ID 559292     -> Index 3
  ID 10116      -> Index 4
  ID 284812     -> Index 5
  ID 83333      -> Index 6
  ID 7227       -> Index 7
  ID 6239       -> Index 8
  ID 83332      -> Index 9
  ID 7955       -> Index 10
  ID 44689      -> Index 11
  ID 39947      -> Index 12
  ID 9913       -> Index 13
  ID 9031       -> Index 14

⚠️ File này KHÔNG có mapping 'original_id_to_species_id'.
👉 Nghĩa là: Model đang học ID gốc thô (Raw ID). E.coli K12 và E.coli 